# 12: Integrated lecture: the North Atlantic Oscillation

We reproduce some of the analyses seen in the following publication: 
- Hurrell, J. W., Kushnir, Y., Ottersen, G. and Visbeck, M.: An Overview of the North Atlantic Oscillation, North Atl. Oscil. Clim. Significance Environ. Impact, 1–35, doi:10.1029/134GM01, 2003.

... but using our own tools and data!

## Import the packages

In [ ]:
# Display the plots in the notebook:
%matplotlib inline
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
import pandas as pd  # new package! this is the package at the base of xarray
from eofs.xarray import Eof  # new package! http://ajdawson.github.io/eofs/index.html
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
import matplotlib.path as mpath
theta = np.linspace(0, 2*np.pi, 100)
map_circle = mpath.Path(np.vstack([np.sin(theta), np.cos(theta)]).T * 0.5 + [0.5, 0.5])

#### Useful functions

In [ ]:
def prepare_plot_coast():
    """This function returns prepared axes for the polar plot.
    
    Usage:
        fig, ax = prepare_plot_grey()
    """
    fig = plt.figure(figsize=(9, 7))
    ax = plt.axes(projection=ccrs.NorthPolarStereo())
    ax.set_extent([-180, 180, 20, 90], ccrs.PlateCarree())
    ax.set_boundary(map_circle, transform=ax.transAxes)
    ax.coastlines(); ax.gridlines();
    return fig, ax

In [ ]:
def prepare_plot_grey():
    """This function returns prepared axes for the polar plot.
    
    Usage:
        fig, ax = prepare_plot_coast()
    """
    fig = plt.figure(figsize=(9, 7))
    ax = plt.axes(projection=ccrs.NorthPolarStereo())
    ax.set_extent([-180, 180, 20, 90], ccrs.PlateCarree())
    ax.set_boundary(map_circle, transform=ax.transAxes)
    ax.add_feature(cartopy.feature.LAND, zorder=0, facecolor='lightgrey', edgecolor='lightgrey')
    ax.gridlines();
    return fig, ax

In [ ]:
def correlation_map(da, ref_ts):
    """This function computes a one-point correlation map"""
    # make an empty array that we will fill
    cor_map = da[0, ...] * 0.
    # loop over lats and lons
    for j in np.arange(len(da.latitude)):
        for i in np.arange(len(da.longitude)):
            # we use the .values attribute because this is much faster
            cor_map.values[j, i] = np.corrcoef(da.values[:, j, i], ref_ts.values)[0, 1]
    return cor_map

## Fig. 01: Seasonal SLP

In [ ]:
slp = xr.open_dataset('ERA-Int-Monthly-SLP.nc').sel(latitude=slice(90, 20)).msl / 100.

In [ ]:
# seasonal averages
slp_sa = slp.groupby('time.season').mean(dim='time')

In [ ]:
fig, ax = prepare_plot_coast()
cs = slp_sa.sel(season='DJF').plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                                            levels=np.arange(990, 1030, 4), cmap='RdBu')

In [ ]:
fig, ax = prepare_plot_coast()
cs = slp_sa.sel(season='JJA').plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                                            levels=np.arange(990, 1030, 4), cmap='RdBu', extend='both')

## Fig. 02: Seasonal surface winds

In [ ]:
ds = xr.open_dataset('ERA-Int-Monthly-UVSLP.nc').sel(latitude=slice(90, 20))
# seasonal averages
u_sa = ds.u10.groupby('time.season').mean(dim='time')
v_sa = ds.v10.groupby('time.season').mean(dim='time')

In [ ]:
fig, ax = prepare_plot_grey()
pu, pv = u_sa.sel(season='DJF')[15::5,::12], v_sa.sel(season='DJF')[15::5,::12]
qv = ax.quiver(pu.longitude, pu.latitude, pu.values, pv.values, transform=ccrs.PlateCarree(),
               scale=150, width=0.003)
plt.title('DJF');

In [ ]:
fig, ax = prepare_plot_grey()
pu, pv = u_sa.sel(season='JJA')[15::5,::12], v_sa.sel(season='JJA')[15::5,::12]
qv = ax.quiver(pu.longitude, pu.latitude, pu.values, pv.values, transform=ccrs.PlateCarree(),
               scale=150, width=0.003)
plt.title('JJA');

## Fig 3: Mean 500 hPa geopotential height

In [ ]:
geop = xr.open_dataset('ERA-Int-Monthly-500hPa-UVZ.nc').sel(latitude=slice(90, 20)).z / 9.8
# seasonal averages
geop_sa = geop.groupby('time.season').mean(dim='time')

In [ ]:
# zonal anomaly
geop_za = geop_sa - geop_sa.mean(dim='longitude')

In [ ]:
fig, ax = prepare_plot_coast()
cs = geop_za.sel(season='DJF').plot.contourf(ax=ax, transform=ccrs.PlateCarree(), 
                                             levels=np.linspace(-160, 160, 9))

In [ ]:
fig, ax = prepare_plot_coast()
cs = geop_za.sel(season='JJA').plot.contourf(ax=ax, transform=ccrs.PlateCarree(), 
                                             levels=np.linspace(-160, 160, 9), extend='both')

## Fig 5: One-Point Correlation

In [ ]:
# seasonal ts
geop_djf = geop.where(geop['time.season'] == 'DJF')
geop_djf = geop_djf.rolling(min_periods=3, center=True, time=3).mean()
geop_djf = geop_djf.groupby('time.year').mean('time')[1:, ...]

In [ ]:
# take the reference geop
geop_ts = geop_djf.sel(latitude=65, longitude=-30, method='nearest')
cor_map = correlation_map(geop_djf, geop_ts)

In [ ]:
fig, ax = prepare_plot_coast()
cs = cor_map.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), 
                           levels=np.linspace(-0.8, 0.8, 9), extend='both')
plt.title('One point correlation map (ref 65°N, 30°W)');

## Fig. 06: EOF analysis 

In [ ]:
# seasonal ts
slp_djf = slp.where(slp['time.season'] == 'DJF')
slp_djf = slp_djf.rolling(min_periods=3, center=True, time=3).mean()
slp_djf = slp_djf.groupby('time.year').mean('time')[1:, ...]
# rename the time coordinate so that eofs is happy
slp_djf = slp_djf.rename({'year':'time'})
# compute anomalies by removing the time-mean.
slp_djf_a = slp_djf - slp_djf.mean(dim='time')

In [ ]:
# Atlantic sector
slp_djf_a_as = slp_djf_a.sel(longitude=slice(-90, 40), latitude=slice(70, 20))
# wgts
wgts = slp_djf_a_as.isel(time=0) * 0. + np.sqrt(np.cos(np.deg2rad(slp_djf_a_as.latitude)).clip(0., 1.))
# solve the EOF
solver = Eof(slp_djf_a_as, weights=wgts)

In [ ]:
# Retrieve the 3 first leading PCs
pcs = solver.pcs(npcs=3, pcscaling=1)
# Get the variance fraction accounted for each EOF
variances = solver.varianceFraction()

In [ ]:
# the maps in fig 6 are the regressions to the leading PC. We can do correlation instead:
cor_map = correlation_map(slp_djf, pcs.sel(mode=0))

In [ ]:
fig, ax = prepare_plot_coast()
cs = cor_map.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), 
                           levels=np.linspace(-0.8, 0.8, 9), extend='both')
plt.title('EOF1 DJF (explained variance: {:2.1f}%)'.format(variances[0].values*100));

## Figs 07, 13, 16: composite anomalies

###  Definitions

In [ ]:
pcs.sel(mode=0).plot();

In [ ]:
pc = pcs.sel(mode=0)
yrs_naop = pc.where(pc > 1).dropna(dim='time').time.values
yrs_naon = pc.where(pc <-1).dropna(dim='time').time.values
print('Years with NAO+:', yrs_naop)
print('Years with NAO-:', yrs_naon)

### Surface winds 

In [ ]:
ds = xr.open_dataset('ERA-Int-Monthly-UVSLP.nc').sel(latitude=slice(90, 20))
# seasonal ts
ds = ds.where(ds['time.season'] == 'DJF')
u_dfj = ds.u10.rolling(min_periods=3, center=True, time=3).mean()
u_dfj = u_dfj.groupby('time.year').mean('time')[1:, ...]
v_dfj = ds.v10.rolling(min_periods=3, center=True, time=3).mean()
v_dfj = v_dfj.groupby('time.year').mean('time')[1:, ...]

In [ ]:
# composites
u_naop = u_dfj.sel(year=yrs_naop).mean(dim='year')
v_naop = v_dfj.sel(year=yrs_naop).mean(dim='year')
u_naon = u_dfj.sel(year=yrs_naon).mean(dim='year')
v_naon = v_dfj.sel(year=yrs_naon).mean(dim='year')
u_compo = u_naop - u_naon
v_compo = v_naop - v_naon

In [ ]:
fig, ax = prepare_plot_grey()
pu, pv = u_compo[15::5,::12], v_compo[15::5,::12]
qv = ax.quiver(pu.longitude, pu.latitude, pu.values, pv.values, transform=ccrs.PlateCarree(),
               scale=150, width=0.003)
plt.title('NAO+ minus NAO-: surface winds  (DJF)');

### Surface temp 

In [ ]:
ds = xr.open_dataset('ERA-Int-Monthly-2mTemp.nc').sel(latitude=slice(90, 20))
# seasonal ts
ds = ds.where(ds['time.season'] == 'DJF')
t_dfj = ds.t2m.rolling(min_periods=3, center=True, time=3).mean()
t_dfj = t_dfj.groupby('time.year').mean('time')[1:, ...]
# composites
t_naop = t_dfj.sel(year=yrs_naop).mean(dim='year')
t_naon = t_dfj.sel(year=yrs_naon).mean(dim='year')
t_compo = t_naop - t_naon

In [ ]:
fig, ax = prepare_plot_coast()
t_compo.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-8, 8, 17))
plt.title('NAO+ minus NAO-: 2m Temp  (DJF)');

### Precipitation

In [ ]:
ds = xr.open_dataset('ERA-Int-Monthly-P.nc').sel(latitude=slice(90, 20))
# seasonal ts
ds = ds.where(ds['time.season'] == 'DJF')
p_dfj = ds.tp.rolling(min_periods=3, center=True, time=3).mean()
p_dfj = p_dfj.groupby('time.year').mean('time')[1:, ...]
# composites
p_naop = p_dfj.sel(year=yrs_naop).mean(dim='year')
p_naon = p_dfj.sel(year=yrs_naon).mean(dim='year')
p_compo = p_naop - p_naon

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
p_compo.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-4, 4, 9), 
                      cbar_kwargs={'label':'mm d $^{-1}$'})
ax.set_extent([-90, 80, 20, 80], ccrs.PlateCarree())
ax.coastlines();
plt.title('NAO+ minus NAO-: precipitation  (DJF)');

## Summary 

Plots from:
- Wanner, H., Brönnimann, S., Casty, C., et al. (2001): North Atlantic oscillation - Concepts and studies. Surv. Geophys., 22(1984):321–382.

**NAO+**

<img src="https://dl.dropboxusercontent.com/u/20930277/do_not_delete/wanner_etal_01_naoplus.jpg" width="80%"  align="left">

**NAO-**

<img src="https://dl.dropboxusercontent.com/u/20930277/do_not_delete/wanner_etal_01_naominus.jpg" width="80%"  align="left">